In [14]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder
from imblearn.over_sampling import SMOTE
import tensorflow as tf
from tensorflow.keras import layers, models
from datetime import datetime
from sklearn.metrics import classification_report


In [ ]:

df = pd.read_csv('credit_card_fraud_dataset.csv')  
df.head()

,TransactionID,TransactionDate,Amount,MerchantID,TransactionType,Location,IsFraud
0,1,2024-04-03 14:15:35.462794,4189.27,688,refund,San Antonio,0
1,2,2024-03-19 13:20:35.462824,2659.71,109,refund,Dallas,0
2,3,2024-01-08 10:08:35.462834,784.00,394,purchase,New York,0
3,4,2024-04-13 23:50:35.462850,3514.40,944,purchase,Philadelphia,0
4,5,2024-07-12 18:51:35.462858,369.07,475,purchase,Phoenix,0


In [ ]:
df = df.drop(['TransactionID', 'TransactionDate'], axis=1)

df['TransactionType'] = LabelEncoder().fit_transform(df['TransactionType'])  
df['Location'] = LabelEncoder().fit_transform(df['Location'])  


X = df.drop('IsFraud', axis=1)
y = df['IsFraud']


scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


In [ ]:

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, stratify=y, random_state=42)


In [ ]:

sm = SMOTE(random_state=42)
X_train_res, y_train_res = sm.fit_resample(X_train, y_train)


In [ ]:

model = models.Sequential([
    layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    layers.Dense(128, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])


In [20]:

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])


In [ ]:

log_dir = "logs/fraud_" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_cb = tf.keras.callbacks.TensorBoard(log_dir=log_dir)


In [ ]:

history = model.fit(X_train_res, y_train_res,
                    epochs=10,
                    batch_size=2048,
                    validation_data=(X_test, y_test),
                    callbacks=[tensorboard_cb])


Epoch 1/10
78/78 [==============================] - 2s 13ms/step - loss: 0.6883 - accuracy: 0.5401 - val_loss: 0.6881 - val_accuracy: 0.4852
Epoch 2/10
78/78 [==============================] - 0s 5ms/step - loss: 0.6822 - accuracy: 0.5649 - val_loss: 0.7165 - val_accuracy: 0.4103
Epoch 3/10
78/78 [==============================] - 0s 5ms/step - loss: 0.6771 - accuracy: 0.5781 - val_loss: 0.6726 - val_accuracy: 0.5721
Epoch 4/10
78/78 [==============================] - 0s 5ms/step - loss: 0.6721 - accuracy: 0.5888 - val_loss: 0.6591 - val_accuracy: 0.5835
Epoch 5/10
78/78 [==============================] - 0s 5ms/step - loss: 0.6657 - accuracy: 0.6035 - val_loss: 0.6326 - val_accuracy: 0.6552
Epoch 6/10
78/78 [==============================] - 1s 7ms/step - loss: 0.6598 - accuracy: 0.6121 - val_loss: 0.6504 - val_accuracy: 0.6345
Epoch 7/10
78/78 [==============================] - 1s 7ms/step - loss: 0.6537 - accuracy: 0.6190 - val_loss: 0.6599 - val_accuracy: 0.5842
Epoch 8/10
78/78 [=

In [ ]:


model_unbalanced = tf.keras.models.clone_model(model)
model_unbalanced.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model_unbalanced.fit(X_train, y_train, epochs=10, batch_size=2048, verbose=0)
y_pred_unbalanced = (model_unbalanced.predict(X_test) > 0.5).astype(int)

y_pred_balanced = (model.predict(X_test) > 0.5).astype(int)

print("Before SMOTE:")
print(classification_report(y_test, y_pred_unbalanced))

print("After SMOTE:")
print(classification_report(y_test, y_pred_balanced))


625/625 [==============================] - 1s 1ms/step
Before SMOTE:
              precision    recall  f1-score   support

           0       0.99      1.00      0.99     19800
           1       0.00      0.00      0.00       200

    accuracy                           0.99     20000
   macro avg       0.49      0.50      0.50     20000
weighted avg       0.98      0.99      0.99     20000

After SMOTE:
              precision    recall  f1-score   support

           0       0.99      0.66      0.79     19800
           1       0.01      0.31      0.02       200

    accuracy                           0.66     20000
   macro avg       0.50      0.49      0.41     20000
weighted avg       0.98      0.66      0.78     20000



c:\Users\ASLAM\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\ASLAM\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\ASLAM\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo